### Trying out RAG with ollama and chromadb to informally access the text material

ollama is recommended over hugging face for local experimentation

it uses a docker-like syntax to pull LLM's

#### Preliminaries

In [2]:
%%capture
!ollama pull nomic-embed-text
!ollama pull llama3.1

In [ ]:
import gradio as gr
import ollama
import chromadb

chromaclient = chromadb.PersistentClient(path="/home/imagery/crc5rag")
collection = chromaclient.get_collection(name="crc5rag")

#### Execute a query with llama3.1 and the supplementary text (RAG)

In [ ]:
def ragask(query):
    # embed the current query
    queryembed = ollama.embed(model="nomic-embed-text", input=query)['embeddings']
    
    # use the embedded current query to retrieve the most relevant document chunks (as text NOT as embeddings)
    relateddocs = '\n\n'.join(collection.query(query_embeddings=queryembed, n_results=4)['documents'][0])

    # generate an answer
    prompt = f"Answer the question: {query} referring to the following text as a resource: {relateddocs}"
    ragoutput = ollama.generate(model="llama3.1", prompt=prompt, stream=False)

    return ragoutput['response']

gr.Interface(fn=ragask, inputs="text", outputs="text").launch(server_name="0.0.0.0", server_port=7860)   